<a href="https://colab.research.google.com/github/jvataidee/PesquisaOperacional/blob/master/analytic_hierarchy_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**by: [João Ataíde](https://www.joaoataide.com)**
# **Métodos de Agregação e Síntese**

---



In [ ]:
#importar bibliotecas
import pandas as pd
import numpy as np
import os
import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations, product

# **Métodos Cardinais**

## **Analytic Hierarchy Process AHP**